In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read in the dataset
df = pd.read_excel("data/ANZ_synthesised_transaction_dataset.xlsx")

In [3]:
# Overview of the dataset
df.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12043 entries, 0 to 12042
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   status             12043 non-null  object        
 1   card_present_flag  7717 non-null   float64       
 2   bpay_biller_code   885 non-null    object        
 3   account            12043 non-null  object        
 4   currency           12043 non-null  object        
 5   long_lat           12043 non-null  object        
 6   txn_description    12043 non-null  object        
 7   merchant_id        7717 non-null   object        
 8   merchant_code      883 non-null    float64       
 9   first_name         12043 non-null  object        
 10  balance            12043 non-null  float64       
 11  date               12043 non-null  datetime64[ns]
 12  gender             12043 non-null  object        
 13  age                12043 non-null  int64         
 14  mercha

In [5]:
# Check if the transaction_ids are unique
df.transaction_id.is_unique

True

The dataset contains 12043 unique transaction entries. Some of the columns have missing data.

In [6]:
# Number of unique customer_ids
df.customer_id.nunique()

100

In [7]:
# Number of unique accounts
df.account.nunique()

100

There are 100 customers in the dataset each of whom has one account.

In [8]:
# Find the earliest date in the dataset
min_date = df.date.min()
min_date

Timestamp('2018-08-01 00:00:00')

In [9]:
# Find the latest date in the dataset
max_date = df.date.max()
max_date

Timestamp('2018-10-31 00:00:00')

In [10]:
df.date.nunique()

91

The earliest and latest dates in the dataset are 1st August 2018 and 31 October 2018, respectively. The dataset contains 91 unique dates while we expect 92 days over August, September and October. Hence there should be one day missing.

A closer look at the data types of each column tells us that we would need to seperate latitudes and longitudes of customers (column long_lat) and merchants (column merchant_long_lat) into seperate columns.

In [11]:
df[["cust_long", "cust_lat"]] = df.long_lat.str.split("-", expand=True)

In [12]:
df[["merchant_long", "merchant_lat"]] = df.merchant_long_lat.str.split("-", expand=True)

In [13]:
# Delete the split columns
del df["long_lat"]
del df["merchant_long_lat"]

In [14]:
# Overview of the dataset after the splits
df.head()

,status,card_present_flag,bpay_biller_code,account,currency,txn_description,merchant_id,merchant_code,first_name,balance,...,extraction,amount,transaction_id,country,customer_id,movement,cust_long,cust_lat,merchant_long,merchant_lat
0,authorized,1.0,NaN,ACC-1598451071,AUD,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,35.39,...,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,debit,153.41,27.95,153.38,27.99
1,authorized,0.0,NaN,ACC-1598451071,AUD,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,21.20,...,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,debit,153.41,27.95,151.21,33.87
2,authorized,1.0,NaN,ACC-1222300524,AUD,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,5.71,...,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,debit,151.23,33.94,151.21,33.87
3,authorized,1.0,NaN,ACC-1037050564,AUD,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,2117.22,...,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,debit,153.10,27.66,153.05,26.68
4,authorized,1.0,NaN,ACC-1598451071,AUD,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,17.95,...,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,debit,153.41,27.95,153.44,28.06


Next we would explore the taxation description column to investigate different kinds of transactions. 

In [15]:
df.txn_description.value_counts()

SALES-POS     3934
POS           3783
PAYMENT       2600
PAY/SALARY     883
INTER BANK     742
PHONE BANK     101
Name: txn_description, dtype: int64

Most of the merchant-related data columns have the same number of non-null values. One assumption is that transactions with null merchant-related data might be inter bank transfer or salary payment. We could verify this assumption by looking at those transactions.

In [16]:
df_null_merchant = df[df.merchant_id.isnull()]
df_null_merchant.txn_description.value_counts()

PAYMENT       2600
PAY/SALARY     883
INTER BANK     742
PHONE BANK     101
Name: txn_description, dtype: int64

As suspected, merchants are not involved for InterBank transfers or Salary payments. Next, we would explore some other categorical columns to see if there is any special record.

In [17]:
df.status.value_counts()

authorized    7717
posted        4326
Name: status, dtype: int64

In [18]:
df.currency.value_counts()

AUD    12043
Name: currency, dtype: int64

In [19]:
df.country.value_counts()

Australia    12043
Name: country, dtype: int64

Check ranges of longitudes and latitudes of customers and merchants to identify any outliers.

In [20]:
df.cust_long.min()

'114.62 '

In [21]:
df.cust_lat.max()

'573.00'

In [22]:
df.merchant_long.min()

TypeError: '<=' not supported between instances of 'str' and 'float'

In [23]:
df.merchant_lat.max()

TypeError: '>=' not supported between instances of 'str' and 'float'